In [2]:
class Descriptor(object):
    def __get__(self, instance, cls):
        print ('in __get__', instance, cls)
        # return instance.__dict__[xxx]
        return instance.__dict__['x']
    
    def __set__(self, instance, value):
        print ('in __set__', instance, cls)
        # instance.__dict__[xxx] = value
        instance.__dict__['x'] = value
    
    def __delete__(self, instance):
        print ('in __del__')
        # del instance.__dict__[xxx]
        
        

In [3]:
class A(object):
    x = Descriptor()

In [4]:
a = A()
a.x
# 对象a被传入__get__方法的instance中, A被传递给相应的cls变量

in __get__ <__main__.A object at 0x107b9c588> <class '__main__.A'>


In [5]:
A.x #直接通过类（而非实例）访问x

in __get__ None <class '__main__.A'>


In [ ]:
a.x = 5 #调用__set__成员函数，5传递给value
del a.x #调用__delete__成员函数

In [7]:
a = A()
a.x
print(a.__dict__)

in __get__ <__main__.A object at 0x107b9ca20> <class '__main__.A'>
{}


In [8]:
a = A()
a.x = 5
print(a.x)

in __set__ <__main__.A object at 0x107b9c8d0> 5
in __get__ <__main__.A object at 0x107b9c8d0> <class '__main__.A'>
None


In [9]:
class Attr(object):
    def __init__(self, name, type_):
        self.name = name
        self.type_ = type_
        
    def __get__(self, instance, cls):
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if not isinstance(value, self.type_):
            raise TypeError('expected value type: %s' % self.type_)
        instance.__dict__[self.name] = value
    
    def __delete__(self, instance):
        del instance.__dict__[self.name]
class Person(object):
    name = Attr('name', str)
    age = Attr('age', int)
    height = Attr('height', float)

In [11]:
p = Person()
p.name = 'Bob'
print(p.name)

Bob


In [12]:
p.age = '17'

TypeError: expected value type: <class 'int'>